In [1]:
import random
import math
import pandas as pd
import numpy as np
from keras.models import model_from_json
from numpy import array

Using TensorFlow backend.


In [2]:
x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12 = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
ufcf = 0.0
dfcf = 0.0
max_D = {}
min_D = {}
max_N = {}
min_N = {}

class boundaries:
    max_D = {}
    min_D = {}
    max_N = {}
    min_N = {}

    def __init__(self, max_D, min_D, max_N, min_N): 
        self.max_D = max_D 
        self.min_D = min_D
        self.max_N = max_N
        self.min_N = min_N

def read_csv():
    #Leyendo archivo
    df = pd.read_csv('Interpolation/InterpolatedDenMonth.csv')
    df_N = pd.read_csv('Interpolation/InterpolatedNumMonth.csv')
    global ufcf, dfcf, max_D, max_N, min_D, min_N
    global x1, x2, x3, x4, x5, x6, x7, x8, x9, x10, x11, x12
    max_D = {'D REVENUE':df['D REVENUE'].max(), 'U CR':df['U CR'].max(), 'D OE':df['D OE'].max(), 
       'D NOI':df['D NOI'].max(),'U CAPEX':df['U CAPEX'].max(), 'U CWK':df['U CWK'].max()} 
    min_D = {'D REVENUE':df['D REVENUE'].min(), 'U CR':df['U CR'].min(), 'D OE':df['D OE'].min(), 
       'D NOI':df['D NOI'].min(),'U CAPEX':df['U CAPEX'].min(), 'U CWK':df['U CWK'].min()}
    max_N = {'U REVENUE':df_N['U REVENUE'].max(), 'D CR':df_N['D CR'].max(), 'U OE':df_N['U OE'].max(), 
       'U NOI':df_N['U NOI'].max(),'D CAPEX':df_N['D CAPEX'].max(), 'D CWK':df_N['D CWK'].max()} 
    min_N = {'U REVENUE':df_N['U REVENUE'].min(), 'D CR':df_N['D CR'].min(), 'U OE':df_N['U OE'].min(), 
       'U NOI':df_N['U NOI'].min(),'D CAPEX':df_N['D CAPEX'].min(), 'D CWK':df_N['D CWK'].min()}
   
    filas_d, columnas_d = df.count()-1, len(df.columns)-1
    dataset_D = df.values
    #Variables a pasar a la funcion generate_population
    dfcf = float(dataset_D[filas_d, columnas_d][1])
    filas_n, columnas_n = df_N.count()-1, len(df_N.columns)-1
    dataset_N = df_N.values
    ufcf = float(dataset_N[filas_d, columnas_d][1])
    
    boundaries_x = boundaries(max_D, min_D, max_N, min_N)
        
    
    ##Calculo de las x
    x1 = df_N['U REVENUE'].corr(df_N['U FCF'])
    x2 = df_N['D CR'].corr(df_N['U FCF'])
    x3 = df_N['U OE'].corr(df_N['U FCF'])
    x4 = df_N['U NOI'].corr(df_N['U FCF'])
    x5 = df_N['D CAPEX'].corr(df_N['U FCF'])
    x6 = df_N['D CWK'].corr(df_N['U FCF'])
    x7 = df['D REVENUE'].corr(df['D FCF'])
    x8 = df['U CR'].corr(df['D FCF'])
    x9 = df['D OE'].corr(df['D FCF'])
    x10 = df['D NOI'].corr(df['D FCF'])
    x11 = df['U CAPEX'].corr(df['D FCF'])
    x12 = df['U CWK'].corr(df['D FCF'])

    return boundaries_x


def generate_population(boundarie, size):
    population = []
    min_N = boundarie.min_N
    max_N = boundarie.max_N
    min_D = boundarie.min_D
    max_D = boundarie.max_D
    for i in range(size):
        individual = {
            "w1": random.uniform(min_N['U REVENUE'], max_N['U REVENUE']),
            "w2": random.uniform(min_N['D CR'], max_N['D CR']),
            "w3": random.uniform(min_N['U OE'], max_N['U OE']),
            "w4": random.uniform(min_N['U NOI'], max_N['U NOI']),
            "w5": random.uniform(min_N['D CAPEX'], max_N['D CAPEX']),
            "w6": random.uniform(min_N['D CWK'], max_N['D CWK']),
            "w7": random.uniform(min_D['D REVENUE'], max_D['D REVENUE']),
            "w8": random.uniform(min_D['U CR'], max_D['U CR']),
            "w9": random.uniform(min_D['D OE'], max_D['D OE']),
            "w10": random.uniform(min_D['D NOI'], max_D['D NOI']),
            "w11": random.uniform(min_D['U CAPEX'], max_D['U CAPEX']),
            "w12": random.uniform(min_D['U CWK'], max_D['U CWK']),
        }    
        population.append(individual)
    return population

def apply_function(individual):
    # Numerador
    # load json and create model
    json_file = open('modelnum.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model_num = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model_num.load_weights("modelnum.h5")
    # Denominador
    json_file = open('modelden.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model_den = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model_den.load_weights("modelden.h5")
    w1 = individual["w1"]
    w2 = individual["w2"]
    w3 = individual["w3"]
    w4 = individual["w4"]
    w5 = individual["w5"]
    w6 = individual["w6"]
    w7 = individual["w7"]
    w8 = individual["w8"]
    w9 = individual["w9"]
    w10 = individual["w10"]
    w11 = individual["w11"]
    w12 = individual["w12"]
    w1_std = (w1-min_N['U REVENUE'])/(max_N['U REVENUE']-min_N['U REVENUE'])
    w2_std = (w2-min_N['D CR'])/(max_N['D CR']-min_N['D CR'])
    w3_std = (w3-min_N['U OE'])/(max_N['U OE']-min_N['U OE'])
    w4_std = (w4-min_N['U NOI'])/(max_N['U NOI']-min_N['U NOI'])
    w5_std = (w5-min_N['D CAPEX'])/(max_N['D CAPEX']-min_N['D CAPEX'])
    w6_std = (w6-min_N['D CWK'])/(max_N['D CWK']-min_N['D CWK'])
    w7_std = (w7-min_D['D REVENUE'])/(max_D['D REVENUE']-min_D['D REVENUE'])
    w8_std = (w8-min_D['U CR'])/(max_D['U CR']-min_D['U CR'])
    w9_std = (w9-min_D['D OE'])/(max_D['D OE']-min_D['D OE'])
    w10_std = (w10-min_D['D NOI'])/(max_D['D NOI']-min_D['D NOI'])
    w11_std = (w11-min_D['U CAPEX'])/(max_D['U CAPEX']-min_D['U CAPEX'])
    w12_std = (w12-min_D['U CWK'])/(max_D['U CWK']-min_D['U CWK'])
    w_numerador = [[w1_std, w2_std, w3_std, w4_std, w5_std, w6_std]]
    w_denominador = [[w7_std, w8_std, w9_std, w10_std, w11_std, w12_std]]
    Xnewnum = array(w_numerador)
    Xnewden = array(w_denominador)
    # make a prediction
    numerador = loaded_model_num.predict(Xnewnum) 
    denominador = loaded_model_den.predict(Xnewden)
    function_costo1 = ufcf/dfcf
    function_costo2 = numerador/denominador
    function_costo2num = function_costo2[0][0]
    function_result = (function_costo2num - function_costo1)/function_costo1
    return function_result

def choice_by_roulette(sorted_population, fitness_sum):
    offset = 0
    normalized_fitness_sum = fitness_sum

    lowest_fitness = apply_function(sorted_population[0])
    if lowest_fitness < 0:
        offset = -lowest_fitness
        normalized_fitness_sum += offset * len(sorted_population)

    draw = random.uniform(0, 1)

    accumulated = 0
    for individual in sorted_population:
        fitness = apply_function(individual) + offset
        probability = fitness / normalized_fitness_sum
        accumulated += probability

        if draw <= accumulated:
            return individual

def sort_population_by_fitness(population):
    return sorted(population, key=apply_function)


def crossover(individual_a, individual_b):
    w1a = individual_a["w1"]
    w2a = individual_a["w2"]
    w3a = individual_a["w3"]
    w4a = individual_a["w4"]
    w5a = individual_a["w5"]
    w6a = individual_a["w6"]
    w7a = individual_a["w7"]
    w8a = individual_a["w8"]
    w9a = individual_a["w9"]
    w10a = individual_a["w10"]
    w11a = individual_a["w11"]
    w12a = individual_a["w12"]
    

    w1b = individual_b["w1"]
    w2b = individual_b["w2"]
    w3b = individual_b["w3"]
    w4b = individual_b["w4"]
    w5b = individual_b["w5"]
    w6b = individual_b["w6"]
    w7b = individual_b["w7"]
    w8b = individual_b["w8"]
    w9b = individual_b["w9"]
    w10b = individual_b["w10"]
    w11b = individual_b["w11"]
    w12b = individual_b["w12"]

    result_crossover = {"w1": (w1a + w1b) / 2, "w2": (w2a + w2b) / 2, "w3": (w3a + w3b) / 2, "w4": (w4a + w4b) / 2, 
    "w5": (w5a + w5b) / 2, "w6": (w6a + w6b) / 2, "w7": (w7a + w7b) / 2, "w8": (w8a + w8b) / 2,
    "w9": (w9a + w9b) / 2, "w10": (w10a + w10b) / 2, "w11": (w11a + w11b) / 2, "w12": (w12a + w12b) / 2}
    return result_crossover


def mutate(individual):
    min_value = -0.05
    max_value = 0.05
    next_w1 = individual["w1"] * (1+random.uniform(min_value, max_value)) 
    next_w2 = individual["w2"] * (1+random.uniform(min_value, max_value))
    next_w3 = individual["w3"] * (1+random.uniform(min_value, max_value))
    next_w4 = individual["w4"] * (1+random.uniform(min_value, max_value))
    next_w5 = individual["w5"] * (1+random.uniform(min_value, max_value))
    next_w6 = individual["w6"] * (1+random.uniform(min_value, max_value))
    next_w7 = individual["w7"] * (1+random.uniform(min_value, max_value))
    next_w8 = individual["w8"] * (1+random.uniform(min_value, max_value))
    next_w9 = individual["w9"] * (1+random.uniform(min_value, max_value))
    next_w10 = individual["w10"] * (1+random.uniform(min_value, max_value))
    next_w11 = individual["w11"] * (1+random.uniform(min_value, max_value))
    next_w12 = individual["w12"] * (1+random.uniform(min_value, max_value))

    result_mutation = {"w1": next_w1, "w2": next_w2, "w3": next_w3, "w4": next_w4, "w5": next_w5,
    "w6": next_w6, "w7": next_w7, "w8": next_w8, "w9": next_w9, "w10": next_w10, "w11": next_w11, "w12": next_w12}
    return result_mutation


def make_next_generation(previous_population):
    next_generation = []
    sorted_by_fitness_population = sort_population_by_fitness(previous_population)
    population_size = len(previous_population)
    fitness_sum = sum(apply_function(individual) for individual in previous_population)

    for i in range(population_size):
        first_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum)
        second_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum)

        individual = crossover(first_choice, second_choice)
        individual = mutate(individual)
        next_generation.append(individual)

    return next_generation

In [3]:
def main():
    generations = 10
    #min_value = 0
    #max_value = 0.85
    #No estoy seguro pero creo necesito verificacion por parte de otro programador gracias
    boundaries_x = read_csv()    
    population = generate_population(boundarie=boundaries_x, size=10)

    i = 1
    while True:
        print(f"🧬 GENERATION {i}")

        for individual in population:
             print(individual, apply_function(individual))

        if i == generations:
            break

        i += 1

        population = make_next_generation(population)

    best_individual = sort_population_by_fitness(population)[-1]
    print("\n🔬 FINAL RESULT")
    print(best_individual, (apply_function(best_individual)*100), "%")

In [4]:
if __name__ == "__main__":
    main()

🧬 GENERATION 1
{'w1': 3127.0958200296373, 'w2': 1071.1496309408935, 'w3': 632.8314985323364, 'w4': 479.91047451568295, 'w5': 391.626003227247, 'w6': 3433.658050609773, 'w7': 2630.842076702549, 'w8': 668.9120114257524, 'w9': 401.8124070854582, 'w10': 638.2842471148821, 'w11': 541.6733091186122, 'w12': 2379.0025897058695} 0.2669684160533922
{'w1': 3037.220814267286, 'w2': 896.5579925989741, 'w3': 386.04449965376233, 'w4': 199.11266770649104, 'w5': 915.13459710795, 'w6': 3536.560807676202, 'w7': 3095.621789038636, 'w8': 878.9417610320814, 'w9': 570.9436656081153, 'w10': 329.9537387418766, 'w11': 787.6108747543935, 'w12': 3027.3303154989862} 0.7361080312858035
{'w1': 2923.612794452684, 'w2': 1499.3914783457274, 'w3': 500.3680465733788, 'w4': 323.58293652333384, 'w5': 910.1871037351394, 'w6': 4452.088593546528, 'w7': 2039.7833811509804, 'w8': 1120.7001457082683, 'w9': 450.2761046945136, 'w10': 673.1427335145453, 'w11': 325.5605571321952, 'w12': 3966.9528357965887} 0.5030921200354336
{'w1': 

{'w1': 2496.2388831552375, 'w2': 1076.9878773907612, 'w3': 419.543903358258, 'w4': 344.6572407527981, 'w5': 751.1222730079951, 'w6': 3110.264647760867, 'w7': 2245.968933619818, 'w8': 889.6568801476767, 'w9': 433.5859549453114, 'w10': 591.3725005539613, 'w11': 281.8792222811008, 'w12': 2902.75890764193} 0.5357291051355275
{'w1': 2516.7154474710196, 'w2': 1122.9296107356115, 'w3': 416.44407653661654, 'w4': 242.22495705050244, 'w5': 557.3922600529975, 'w6': 3146.819701739558, 'w7': 2122.71193223782, 'w8': 1355.1439768870616, 'w9': 607.8082664310526, 'w10': 437.77111433526835, 'w11': 539.9592519008279, 'w12': 3482.641915241087} 0.5494611597400356
{'w1': 2540.520430277468, 'w2': 1103.5363482778307, 'w3': 411.55816941144207, 'w4': 281.1620374266163, 'w5': 587.2200656377488, 'w6': 3040.736099074314, 'w7': 2383.5519840015104, 'w8': 1167.9793576195946, 'w9': 497.0395777963667, 'w10': 458.97697347985934, 'w11': 552.6658069289509, 'w12': 3652.3999607004835} 0.5624110531299064
{'w1': 2772.11923782

🧬 GENERATION 6
{'w1': 2289.1328282448917, 'w2': 1202.014578650326, 'w3': 433.26450763587354, 'w4': 269.41649581132185, 'w5': 674.731436560053, 'w6': 3120.3327868304254, 'w7': 1984.5812043122949, 'w8': 1188.6243084350342, 'w9': 601.0683068656818, 'w10': 493.2230638492173, 'w11': 319.92070840470507, 'w12': 3277.576861975325} 0.5039784386561115
{'w1': 2474.0515413720036, 'w2': 1171.5095145131677, 'w3': 419.61005311291166, 'w4': 234.76904460261704, 'w5': 588.910282545673, 'w6': 2954.439929252269, 'w7': 2017.3669824071653, 'w8': 1325.6534006581603, 'w9': 599.4114198861321, 'w10': 431.2467091293588, 'w11': 457.9091030198853, 'w12': 3480.5236665338057} 0.5831235120494463
{'w1': 2392.0022539449483, 'w2': 1052.9713418989143, 'w3': 449.319028091918, 'w4': 270.8692496740894, 'w5': 675.2115491025537, 'w6': 3238.9862934579905, 'w7': 2030.146505169495, 'w8': 1259.4662334968216, 'w9': 615.790034124886, 'w10': 504.12787712606803, 'w11': 319.9595270123864, 'w12': 3315.2567359966056} 0.5053446658800427


{'w1': 2443.215930487945, 'w2': 1079.6876385086962, 'w3': 463.147558657763, 'w4': 242.92906991105932, 'w5': 653.6410374771064, 'w6': 3108.0267083166386, 'w7': 1923.2898357390552, 'w8': 1259.002284261733, 'w9': 603.1948195564053, 'w10': 477.7267623966741, 'w11': 367.79071502886757, 'w12': 3384.607293211277} 0.5288690650776247
{'w1': 2532.6297210017137, 'w2': 1141.7819870172589, 'w3': 469.9849607207929, 'w4': 236.5186297879362, 'w5': 641.2935627688738, 'w6': 3016.8034389974573, 'w7': 2014.3054985243007, 'w8': 1293.5999871878578, 'w9': 573.1572737673299, 'w10': 473.1509831912692, 'w11': 380.5158334387851, 'w12': 3468.273045353206} 0.5781460474265894
{'w1': 2471.9595128593137, 'w2': 1130.6543660235113, 'w3': 447.4535019285165, 'w4': 217.20376018973272, 'w5': 615.9317500530951, 'w6': 2833.170056240276, 'w7': 1974.0035268345155, 'w8': 1320.031491488261, 'w9': 620.8793698640565, 'w10': 445.1301652135617, 'w11': 401.74519583111527, 'w12': 3403.232233384321} 0.5923716367766673
{'w1': 2602.83365


🔬 FINAL RESULT
{'w1': 2656.9116535040007, 'w2': 1131.4592387474688, 'w3': 436.81247290743556, 'w4': 249.36365227987588, 'w5': 636.1247982632042, 'w6': 2910.140501957674, 'w7': 2040.3003549194361, 'w8': 1409.9595837823522, 'w9': 592.5299335870224, 'w10': 506.54907298746235, 'w11': 405.6005962926721, 'w12': 3414.180583813888} 63.500990065202146 %
